In [3]:
from brian2 import *
import NeuronEquations
import BrianHF
from ipywidgets import interact
import numpy as np
from bimvee.importAe import importAe
from EvCamDatabase import DAVIS_346B

In [4]:
@interact(tau=(1, 100), vt=(0, 30), vr=(-10, 10), v0=(-10, 30), gL=(0, 10),
          K_GABA=(0, 10), K_AMPA=(0, 100), taugd=(0, 10, 0.25), tauad=(0, 10, 0.25),
          taugr=(0, 10, 0.25), tauar=(0, 10, 0.25), w_N = (0, 100), w_G = (0, 100),
          w_A = (0, 100))



def simulate_neurons(tau=10, vt=15, vr=0, v0=20, gL=1, K_GABA=1, K_AMPA=10, taugd=5,
                     tauad=2, taugr=0.25, tauar=0.5, w_N=80, w_G=30, w_A=50):
    
    # XXX: Extract the event stream using bimvee - needs refactoring to be more general
    grid_width, grid_height= DAVIS_346B['width'], DAVIS_346B['height']
    filePath = 'MVSEC_short_outdoor'
    events = importAe(filePathOrName=filePath)
    eventStream = events[0]['data']['left']['dvs']
    eventStream.popitem()
    
    
    
    # Brian Simulation Scope
    start_scope()
    '''
        Steps for the setting up the network:
            1. Generate the input spikes
            2. Set the parameters for the network (neurons and synapses)
            3. Create the neuron group(s)
            4. Create the synapses
            5. Connect the synapses
            6. Define the weights of the synapses
            7. Set up monitors according to need
            8. Run the simulation
    '''
        
    # 1. Generate the input spikes from the event stream
    # HACK XXX: The input is now not in real time, must be fixed eventually to process real time event data
    inputSpikes = BrianHF.event_to_spike(eventStream, grid_width, grid_height)
    
    
    # 2. Set the parameters for the network (neurons and synapses)
    # Neuron Parameters
    N_Neurons = grid_width * grid_height    # Number of neurons
    tau = tau*ms    # Time constant
    vt = vt*mV    # Threshold Voltage
    vr = vr*mV    # Reset Voltage
    v0 = v0*mV    # Resting Voltage - Not necessarily used
    gL = gL/ms    # Leak Conductance - Not necessarily used
    
    '''
        Neighborhood Size (num_Neighbors) - Affects the number of neighbors a central neuron based on the L1 Distance
        Neighboring Neurons --> (abs(X_pre - X_post) <= num_Neighbors  and abs(Y_pre - Y_post) <= num_Neighbors)
    '''
    num_Neighbors = 1

    Eqs_Neurons = NeuronEquations.EQ_LIF    # Neuron Equations

    # Synapse Parameters
    K_GABA = K_GABA
    K_AMPA = K_AMPA
    taugd = taugd * ms
    tauad = tauad * ms
    taugr = taugr * ms
    tauar = tauar * ms


    # 3. Create the neuron group(s)
    neuronsGrid = NeuronGroup(N_Neurons, Eqs_Neurons, threshold='v>vt', reset='v = vr', refractory=1 * ms, method='euler')

    # FIXME: Verify the grid coordinates and assign the X and Y values to the neurons accordingly
    # Generate x and y values for each neuron
    x_values = np.repeat(np.arange(grid_width), grid_height)
    y_values = np.tile(np.arange(grid_height), grid_width)
    neuronsGrid.X = x_values
    neuronsGrid.Y = y_values
    
    
    # 4. Create the synapses
    Syn_Input_Neurons = Synapses(inputSpikes, neuronsGrid, 'w : volt/second', on_pre='Ia1 += w')
    Syn_Neurons_GABA = Synapses(neuronsGrid, neuronsGrid, 'w : volt/second', on_pre='Ig1 -= w')
    Syn_Neurons_AMPA = Synapses(neuronsGrid, neuronsGrid, 'w : volt/second', on_pre='Ia1 += w')


    # 5. Connect the synapses
    Syn_Input_Neurons.connect(condition= 'i==j')    # Connect the first synapses from input to G_neurons on a 1 to 1 basis
    Syn_Neurons_GABA.connect(condition='i != j and abs(X_pre - X_post) <= num_Neighbors and abs(Y_pre - Y_post) <= num_Neighbors')    # Connect the second group of synapses from a neuron to its neighbors
    Syn_Neurons_AMPA.connect(condition='i == j')     # Connect the last set of synapses from a neuron to itself (recurrent)


    # 6. Define the weights of the synapses
    Syn_Input_Neurons.w = w_N * volt/second
    Syn_Neurons_GABA.w = w_G * volt/second
    Syn_Neurons_AMPA.w = w_A * volt/second
    
    
    # 7. Set up monitors
    SpikeMon_Input = SpikeMonitor(inputSpikes)    # Monitor the spikes from the input
    SpikeMon_Neurons = SpikeMonitor(neuronsGrid)    # Monitor the spikes from the neuron grid
    StateMon_Neurons = StateMonitor(neuronsGrid, variables=True, record=True)    # Monitor the state variables - True for all variables


    # 8. Run the simulation
    run(100*ms)

    # VISUALISATION
    BrianHF.visualise_spikes([SpikeMon_Input], figSize=(12,3), figTitle='Input Spikes')
    BrianHF.visualise_spikes([SpikeMon_Neurons], figSize=(12,3), figTitle='Output Spikes')
    
    # Ploting the interspike interval
    BrianHF.visualise_interSpikeInterval(SpikeMon_Neurons, [5])
    BrianHF.visualise_neurons_states(StateMon_Neurons, [4, 5], 'all')    
    

interactive(children=(IntSlider(value=10, description='tau', min=1), IntSlider(value=15, description='vt', max…